In [1]:
%cd ..

G:\My Drive\0-Research-Project\MedStar\MS_CODE\FieldNN


# Next PreCode

## EmbedBlock

In [2]:
import os
from recfldgrn.datapoint import load_df_data_from_folder
from fieldnn.utils.layerfn import traverse, convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
# l

recfldgrn_list =['P@age-AgeNumeDftGrn',
                 'P@basicInfo-basicInfoDftGrn',
                 
                 'EC@BasicInfo-BasicDftGrn',
                 'EC@DT_min-DTDftGrn',
                 
                 'A1C@DT-DTDftGrn',
                 'A1C@V-A1CNumeDftGrn',
                 
                 'Diag@DT-DTDftGrn',
                 'Diag@Value-DiagDftGrn',
                 
                 'PNSectSent@Sentence-Tk@TknzLLMGrn']

data/ProcData/TensorFolder/Task2YearXXX


In [3]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)
    print(tensor_folder)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

data/ProcData/TensorFolder/Task2YearXXX\P@age-AgeNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX\P@basicInfo-basicInfoDftGrn
data/ProcData/TensorFolder/Task2YearXXX\EC@BasicInfo-BasicDftGrn
data/ProcData/TensorFolder/Task2YearXXX\EC@DT_min-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX\A1C@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX\A1C@V-A1CNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX\Diag@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX\Diag@Value-DiagDftGrn
data/ProcData/TensorFolder/Task2YearXXX\PNSectSent@Sentence-Tk@TknzLLMGrn
B-P@age-AgeNumeDftGrn_wgt (4, 19)
B-P@basicInfo-basicInfoDftGrn_idx (4, 2)
B-P-EC@BasicInfo-BasicDftGrn_idx (4, 25, 2)
B-P-EC@DT_min-DTDftGrn_idx (4, 25, 7)
B-P-EC-A1C@DT-DTDftGrn_idx (4, 25, 1, 7)
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


In [4]:
full_recfldgrn_list = [k for k in batch_rfg]
full_recfldgrn_list

['B-P@age-AgeNumeDftGrn_wgt',
 'B-P@basicInfo-basicInfoDftGrn_idx',
 'B-P-EC@BasicInfo-BasicDftGrn_idx',
 'B-P-EC@DT_min-DTDftGrn_idx',
 'B-P-EC-A1C@DT-DTDftGrn_idx',
 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [5]:
import torch 

RECFLD_TO_TENSOR = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
   
    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    info_dict = {'holder': holder, 'holder_wgt': holder_wgt}
    
    RECFLD_TO_TENSOR[full_recfldgrn] = info_dict
    

In [6]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_DataFlow

from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


default_SubUnitName = 'E'
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 




df_SubUnit = get_EmbeddingBlock_DataFlow(full_recfldgrn_list, default_SubUnitName)

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)
df_SubUnit['SubUnit_BasicNN_List'] = s
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s



s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P@age-AgeNumeDftGrn_wgt],B-P@age-AgeNumeDft,3,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
1,E,[B-P@basicInfo-basicInfoDftGrn_idx],B-P@basicInfo-basicInfoDft,3,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, 'Male': 1...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
2,E,[B-P-EC@BasicInfo-BasicDftGrn_idx],B-P-EC@BasicInfo-BasicDft,4,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, 'X': 1, '...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
3,E,[B-P-EC@DT_min-DTDftGrn_idx],B-P-EC@DT_min-DTDft,4,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
4,E,[B-P-EC-A1C@DT-DTDftGrn_idx],B-P-EC-A1C@DT-DTDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
5,E,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
6,E,[B-P-EC-Diag@DT-DTDftGrn_idx],B-P-EC-Diag@DT-DTDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
7,E,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
8,E,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,[expander-LLMEmbed],[{'vocab_tokenizer': PreTrainedTokenizerFast(n...,"[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


In [7]:
from fieldnn.module.embedblock import EmbedBlockLayer


df_Embed_SubUnit = df_SubUnit
EmbedBlock = EmbedBlockLayer(df_SubUnit)

RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)

for k, v in RECFLD_TO_EMBEDTESNOR.items():
    print(k, v['info'].shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


## ReprBlock

In [8]:
full_recfldgrn_list = [i for i in RECFLD_TO_EMBEDTESNOR]
full_recfldgrn_list

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [9]:
from fieldnn.dataflowfn.reprflowfn import get_Repr_dataflow_table
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow_completename
from fieldnn.dataflowfn.reprflowfn import get_Repr_SubUnit_List

df_dataflow = get_Repr_dataflow_table(full_recfldgrn_list)
# df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'ReducerOnly')
# df_dataflow_new

df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'Reducer&Merger')
df_dataflow = df_dataflow_new.copy()
df_SubUnit = get_Repr_SubUnit_List(df_dataflow)
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid
0,R,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6
1,R,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5
2,R,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4
3,R,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4
4,R,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4
5,R,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4
6,R,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4
7,MR,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4
8,MR,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4
9,R,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3


In [10]:
from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import generate_BasicNN_Config
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List


############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)


df_SubUnit['SubUnit_BasicNN_List'] = s
# df_SubUnit


fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 


s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################

s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

s

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,R,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
1,R,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
2,R,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
3,R,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
4,R,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
5,R,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
6,R,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
7,MR,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4,"[merger-Merger, reducer-Max]","[{}, {}]","[{'nn_type_nn_name': 'merger-Merger', 'Basic_C..."
8,MR,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4,"[merger-Merger, reducer-Max]","[{}, {}]","[{'nn_type_nn_name': 'merger-Merger', 'Basic_C..."
9,R,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."


In [11]:
from fieldnn.module.reprblock import ReprBlockLayer

df_Repr_SubUnit = df_SubUnit
ReprBlock = ReprBlockLayer(df_SubUnit)

In [12]:
fld_updates_dict = {}
for i in RECFLD_TO_EMBEDTESNOR:
    layernum = len(i.split('-'))
    fld = i.split('-')[-1]
    if '@' in fld:
        # print(fld)
        neat_i = '-'.join(i.split('-')[:-1]) + '-' + fld.split('@')[0]
        # print(neat_i)
        same_neat_list = [t for t in RECFLD_TO_EMBEDTESNOR if neat_i + '@' in t]
        # print(same_neat_list)
        if len(same_neat_list) == 1: # itself
            # RECFLD_TO_EMBEDTESNOR[]
            fld_updates_dict[i] = neat_i
            
for old, new in fld_updates_dict.items():
    RECFLD_TO_EMBEDTESNOR[new] = RECFLD_TO_EMBEDTESNOR.pop(old)
    
    
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk']

In [13]:
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk']

In [14]:
OUTPUT_TO_TENSOR = ReprBlock(RECFLD_TO_EMBEDTESNOR)
[i for i in OUTPUT_TO_TENSOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk',
 'B-P-EC-PN-PNSect-PNSectSent',
 'B-P-EC-PN-PNSect',
 'B-P-EC-A1C@DT',
 'B-P-EC-A1C@V',
 'B-P-EC-Diag@DT',
 'B-P-EC-Diag@Value',
 'B-P-EC-PN',
 'B-P-EC-A1C',
 'B-P-EC-Diag',
 'B-P-EC@BasicInfo',
 'B-P-EC@DT_min',
 'B-P-EC@A1C',
 'B-P-EC@Diag',
 'B-P-EC@PN',
 'B-P-EC',
 'B-P@age',
 'B-P@basicInfo',
 'B-P@EC',
 'B-P']

# Output: BasicNN Name

In [15]:
import torch

class PatientLevelPredictionModel(torch.nn.Module):
    """ Naive softmax-layer """
    def __init__(self, df_Embed_SubUnit, df_Repr_SubUnit, 
                 actn_fn_name, loss_fn_name,
                 output_name, embed_size, output_size):
        
        '''
            df_Embed_SubUnit: df_SubUnit for the Embed Block.
            df_Repr_SubUnit: df_Repr_SubUnit for the Repr Block.
            actn_fn_name: like sigmoid, softmax
            loss_fn_name: like CrossEntropy, BCELoss.
            output_name: the featvec name.
            embed_size: the featvec dim.
            output_size: the output dim size. if actn_fn is softmax, then label sets. if actn_fn is sigmoid, then 1. 
            
        '''
        super(PatientLevelPredictionModel, self).__init__()
        
        self.output_name = output_name
        
        self.EmbedBlock = EmbedBlockLayer(df_Embed_SubUnit)
        self.ReprBlock  = ReprBlockLayer(df_Repr_SubUnit)
        self.OutputBlock = torch.nn.Linear(embed_size, output_size)
        
        self.actn_fn_name = actn_fn_name
        if self.actn_fn_name == 'Sigmoid':
            self.actn_method = torch.nn.Sigmoid()
            self.actn_fn = lambda outputvecs: self.actn_method(outputvecs) # will return probs 
        elif self.actn_fn_name == 'Softmax':
            self.actn_method = torch.nn.Softmax()
            self.actn_fn = lambda outputvecs: self.actn_method(outputvecs, dim = 1) # will return probs 
        else:
            raise ValueError(f'Activation Function Name {actn_fn_name} is not available yet')
        
        self.loss_fn_name = loss_fn_name
        if self.loss_fn_name == 'BCELoss':
            assert self.actn_fn_name == 'Sigmoid'
            self.loss_method = torch.nn.BCELoss()
            self.loss_fn = lambda probs, targets: self.loss_method(probs, targets) # will return loss
        
        elif self.loss_fn_name == 'CrossEntropyLoss':
            assert self.actn_fn_name == 'Softmax'
            self.loss_method = torch.nn.CrossEntropyLoss()
            self.loss_fn = lambda probs, targets: self.loss_method(probs, targets) # will return loss
        
        else:
            raise ValueError(f'Loss Function Name {loss_fn_name} is not available yet')
        

    def get_REPR_TENSOR(self, batch_rfg):
        # get the full_recfldgrn_list
        full_recfldgrn_list = [i for i in batch_rfg]

        # prepare RECFLD_TO_TENSOR
        RECFLD_TO_TENSOR = {}
        for full_recfldgrn in full_recfldgrn_list:
            # (1) get the info_raw from batch_rfg
            info_raw = batch_rfg[full_recfldgrn]

            # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
            if '_idx' in full_recfldgrn:
                holder_wgt = 'Empty'
                holder = torch.LongTensor(info_raw)
            elif '_wgt' in full_recfldgrn:
                holder_wgt = torch.FloatTensor(info_raw)
                # ATTENTION: here holder_wgt could contain zeros in some valid positions.
                holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
            else:
                raise ValueError(f'Invalid suffix "{suffix}"')

            info_dict = {'holder': holder, 'holder_wgt': holder_wgt}
            RECFLD_TO_TENSOR[full_recfldgrn] = info_dict
    
        # get RECLD_TO_EMBEDTENSOR from RECFLD_TO_TENSOR and EmbedBlock
        RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)
        
        # update the full_recfldgrn_list
        full_recfldgrn_list = [i for i in RECFLD_TO_EMBEDTESNOR]

        # update the names of full_recfldgrn_list
        fld_updates_dict = {}
        for i in RECFLD_TO_EMBEDTESNOR:
            layernum = len(i.split('-'))
            fld = i.split('-')[-1]
            if '@' not in fld: continue
            
            # print(fld)
            neat_i = '-'.join(i.split('-')[:-1]) + '-' + fld.split('@')[0]
            # print(neat_i)
            same_neat_list = [t for t in RECFLD_TO_EMBEDTESNOR if neat_i + '@' in t]
            # print(same_neat_list)
            if len(same_neat_list) == 1: fld_updates_dict[i] = neat_i

        for old, new in fld_updates_dict.items():
            RECFLD_TO_EMBEDTESNOR[new] = RECFLD_TO_EMBEDTESNOR.pop(old)
            
        # get the OUTPUT_TO_TENSOR data holder
        REPR_TENSOR = ReprBlock(RECFLD_TO_EMBEDTESNOR)
        return REPR_TENSOR

    def loss_funciton(batch_rfg, batch_targets):
        # batch -> Embed -> Repr -> repr tensor
        REPR_TENSOR = self.get_REPR_TENSOR(batch_rfg)
        info_dict = REPR_TENSOR[self.output_name]
        featvecs = info_dict['info']
        
        outputvecs = self.OutputBlock(featvecs)
        
        probs = self.actn_fn(outputvecs) 
        l = self.loss_fn(probs, batch_targets)
        return l
        
    def forward(self, batch_rfg):
        # forward is the method for the inference.
        # batch -> Embed -> Repr -> repr tensor
        REPR_TENSOR = self.get_REPR_TENSOR(batch_rfg)
        info_dict = REPR_TENSOR[self.output_name]
        featvecs = info_dict['info']
        
        outputvecs = self.OutputBlock(featvecs)
        
        probs = self.actn_fn(outputvecs) 
        return probs

In [16]:
df_Embed_SubUnit = df_Embed_SubUnit
df_Repr_SubUnit = df_Repr_SubUnit
output_name = 'B-P'
embed_size = embed_size
output_size = 1
actn_fn_name = 'Sigmoid' # torch.nn.Sigmoid()
loss_fn_name = 'BCELoss' # torch.nn.BCELoss()

In [17]:
Model = PatientLevelPredictionModel(df_Embed_SubUnit, df_Repr_SubUnit, 
                                    actn_fn_name, loss_fn_name,
                                    output_name, embed_size, output_size)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
# input
[i for i in batch_rfg]

['B-P@age-AgeNumeDftGrn_wgt',
 'B-P@basicInfo-basicInfoDftGrn_idx',
 'B-P-EC@BasicInfo-BasicDftGrn_idx',
 'B-P-EC@DT_min-DTDftGrn_idx',
 'B-P-EC-A1C@DT-DTDftGrn_idx',
 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [19]:
# REPR output

REPR_TENSOR = Model.get_REPR_TENSOR(batch_rfg)

[i for i in REPR_TENSOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk',
 'B-P-EC-PN-PNSect-PNSectSent',
 'B-P-EC-PN-PNSect',
 'B-P-EC-A1C@DT',
 'B-P-EC-A1C@V',
 'B-P-EC-Diag@DT',
 'B-P-EC-Diag@Value',
 'B-P-EC-PN',
 'B-P-EC-A1C',
 'B-P-EC-Diag',
 'B-P-EC@BasicInfo',
 'B-P-EC@DT_min',
 'B-P-EC@A1C',
 'B-P-EC@Diag',
 'B-P-EC@PN',
 'B-P-EC',
 'B-P@age',
 'B-P@basicInfo',
 'B-P@EC',
 'B-P']

In [20]:
Model.output_name

'B-P'

In [21]:
probs = Model(batch_rfg)
probs

tensor([[0.4177],
        [0.7388],
        [0.4037],
        [0.4662]], grad_fn=<SigmoidBackward0>)